In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [14]:
#def load_data():
global smart
global gas
global weather
df =pd.read_csv("/datc/opschaler/smartmeter_data/P01S01W0373.csv",header=0,delimiter=";",parse_dates=['Timestamp', 'gasTimestamp'])

# Split the smartmeter and gasmeter data
smart = df.iloc[:,:7]
gas = df.iloc[:, 7:]

#Read in weather data
weather_path = "//datc//opschaler//weather_data//20170101to20180101.txt"
weather_column_names = ['STN','YYYYMMDD','HH','DD','FH','FF','FX','T','T10','TD','SQ','Q','DR','RH','P','VV','N','U','WW','IX','M','R','S','O','Y']
weather = pd.read_csv(weather_path, delimiter=',', comment='#', names=weather_column_names, parse_dates=['YYYYMMDD', 'HH'])

weather['T10'] = weather['T10'].str.strip()#remove blank spaces in the whole column
weather['T10'].replace('',0,inplace=True)#replace empty gaps with 0 value(int)
weather['T10']=weather.T10.astype(float)

weather['T'] = weather['T']*0.1 # Convert temperature to celcius instead of 0.1 celcius
weather['HH'] = weather['HH']+':00:00'
weather['HH'] = weather['HH'].str.strip() # removes the blank spaces infront of the strings
weather['HH'] = weather['HH'].str.replace('24:00:00', '23:59:59') # KNMI uses 24 for 00 hours.... to have to convert this for to_datetime to be able to work
weather.loc[:,'datetime'] = pd.to_datetime(weather['YYYYMMDD'].astype(str)+' '+weather['HH'].astype(str)) #create datetime column


In [9]:
"""
Problem: How to merge 3 frames with 3 different datetime's?
The format is the same, but they don't have ''exact'' matches with each other. 
i.e. 

"""
#load_data()

# Change all datetime columns to the same column name
smart = smart.rename(index=str, columns={"Timestamp":"datetime"})
gas = gas.rename(index=str, columns={"gasTimestamp":"datetime"})


#df = df[['datetime', 'eMeter', 'eMeterLow', 'ePower', 'gasMeter', 'T', 'Q']]
#df = df[['datetime', 'eMeter', 'eMeterLow', 'ePower', 'gasMeter']]
smart.head()

del weather['STN'], weather['HH'], weather['YYYYMMDD']

# Set datetime as index
weather = weather.set_index(['datetime'])
smart = smart.set_index(['datetime'])
gas = gas.set_index(['datetime'])
print(weather.T10)

datetime
2017-01-01 01:00:00     0.0
2017-01-01 02:00:00     0.0
2017-01-01 03:00:00     0.0
2017-01-01 04:00:00     0.0
2017-01-01 05:00:00     0.0
2017-01-01 06:00:00     6.0
2017-01-01 07:00:00     0.0
2017-01-01 08:00:00     0.0
2017-01-01 09:00:00     0.0
2017-01-01 10:00:00     0.0
2017-01-01 11:00:00     0.0
2017-01-01 12:00:00    -5.0
2017-01-01 13:00:00     0.0
2017-01-01 14:00:00     0.0
2017-01-01 15:00:00     0.0
2017-01-01 16:00:00     0.0
2017-01-01 17:00:00     0.0
2017-01-01 18:00:00    -6.0
2017-01-01 19:00:00     0.0
2017-01-01 20:00:00     0.0
2017-01-01 21:00:00     0.0
2017-01-01 22:00:00     0.0
2017-01-01 23:00:00     0.0
2017-01-01 23:59:59    10.0
2017-01-02 01:00:00     0.0
2017-01-02 02:00:00     0.0
2017-01-02 03:00:00     0.0
2017-01-02 04:00:00     0.0
2017-01-02 05:00:00     0.0
2017-01-02 06:00:00   -35.0
                       ... 
2017-12-31 19:00:00     0.0
2017-12-31 20:00:00     0.0
2017-12-31 21:00:00     0.0
2017-12-31 22:00:00     0.0
2017-12-31 

In [10]:
"""
Resample the dataframes to 10s intervals.
https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.resample.html
https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.interpolate.html#pandas.DataFrame.interpolate
"""
############################
weather = weather.resample('10s').interpolate(method='time')
############################
smart = smart.resample('10s').mean()
# gas contains duplicates, they need to be removed
#gas = gas.drop_duplicates()
gas = gas.resample('H').mean() # Does not look nice, but it works

# Create a ''gasPower'' column. This equals the gas used in that hour
gas['gasPower'] = gas['gasMeter'].diff()
# First entry is NaN, replace NaN by second entry
gas['gasPower'][0] = gas['gasPower'][1]
gas = gas.resample('10s').interpolate(method='time')

In [11]:
df = pd.merge(smart, weather,left_index=True, right_index=True)
df = pd.merge(df, gas,left_index=True, right_index=True)
df.head()

,eMeter,eMeterReturn,eMeterLow,eMeterLowReturn,ePower,ePowerReturn,DD,FH,FF,FX,...,U,WW,IX,M,R,S,O,Y,gasMeter,gasPower
datetime,,,,,,,,,,,,,,,,,,,,,
2017-03-07 18:14:20,3717.472,0.0,3883.382,0.0,376.0,0.0,255.222222,30.0,32.388889,60.0,...,81.761111,NaN,5.0,0.0,0.0,0.0,0.0,0.0,1599.838344,0.194
2017-03-07 18:14:30,3717.473,0.0,3883.382,0.0,381.0,0.0,255.166667,30.0,32.416667,60.0,...,81.758333,NaN,5.0,0.0,0.0,0.0,0.0,0.0,1599.838883,0.194
2017-03-07 18:14:40,3717.474,0.0,3883.382,0.0,378.0,0.0,255.111111,30.0,32.444444,60.0,...,81.755556,NaN,5.0,0.0,0.0,0.0,0.0,0.0,1599.839422,0.194
2017-03-07 18:14:50,3717.475,0.0,3883.382,0.0,376.0,0.0,255.055556,30.0,32.472222,60.0,...,81.752778,NaN,5.0,0.0,0.0,0.0,0.0,0.0,1599.839961,0.194
2017-03-07 18:15:00,3717.476,0.0,3883.382,0.0,380.0,0.0,255.000000,30.0,32.500000,60.0,...,81.750000,NaN,5.0,0.0,0.0,0.0,0.0,0.0,1599.840500,0.194


In [12]:
df.dtypes

eMeter             float64
eMeterReturn       float64
eMeterLow          float64
eMeterLowReturn    float64
ePower             float64
ePowerReturn       float64
DD                 float64
FH                 float64
FF                 float64
FX                 float64
T                  float64
T10                float64
TD                 float64
SQ                 float64
Q                  float64
DR                 float64
RH                 float64
P                  float64
VV                 float64
N                   object
U                  float64
WW                  object
IX                 float64
M                  float64
R                  float64
S                  float64
O                  float64
Y                  float64
gasMeter           float64
gasPower           float64
dtype: object

In [22]:
print(df[11])

KeyError: 11